# Examples and simple tests with HEEPS

This notebook is intended to illustrate and verify basic properties of perfect vortex coronagraphs: the CVC and the RAVC.

## 1. Perfect Classical Vortex Coronagraph with a circular aperture

In [1]:
%matplotlib widget
import proper
from heeps.config import conf
from heeps.pupil import pupil
from heeps.coronagraphs import apodization, vortex, lyotstop, lyot

import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from astropy.io import fits 

import heeps.util.img_processing as impro

In [4]:
'''
Gridsize is very small on-purpose to be able to examine pixel-wise.
This is not intended for a real e2e simulation, just for illustration

conf['pupil_file'] = '', in order to have a proper circular pupil
'''
conf['gridsize']=32
conf['diam']=10
conf['lam'] = 2.2e-6 
conf['pscale'] = 11.4
conf['R_obstr'] = 0 
conf['get_amp'] =True
conf['get_phase'] = True
conf['spiders_width'] = False
conf['pupil_file'] = ''


wf, pup_amp, pup_phase = pupil(conf)
vortex(wf, conf)
before_lyot = impro.crop_img(proper.prop_get_amplitude(wf), conf['npupil'])

npupil = 9


In [7]:
plt.figure()
plt.subplot(121)
plt.imshow(pup_amp)
plt.title('Intensity in the \n pupil plane')

plt.subplot(122)
plt.imshow(before_lyot)
plt.title('Intensity in the Lyot plane \n (before Lyot stop)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Intensity in the Lyot plane \n (before Lyot stop)')

## 2. Perfect Ring Apodized Vortex Coronagraph

In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from astropy.io import fits 
from astropy.visualization import imshow_norm,\
    SqrtStretch, MinMaxInterval, PercentileInterval, ManualInterval,\
    LinearStretch, SinhStretch, LogStretch
from heeps.config import conf
from heeps.pupil import pupil
from heeps.aberrations import wavefront_aberrations
from heeps.coronagraphs import apodization, vortex, lyotstop, lyot
from heeps.detector import detector
import heeps.util.img_processing as impro
from copy import deepcopy
import proper

In [2]:
conf['spiders_width'] = 0.0
conf['LS_params'] = [0.98, 0.03, 0]  
conf['get_amp'] = True
conf['get_phase'] = True
conf['hfov'] = 0.666   
conf['pupil_file'] = ''

wf_start, pup_amp, pup_phase = pupil(conf)    
wf = deepcopy(wf_start)

wf, apo_amp, apo_phase = apodization(wf, conf, RAVC=True)
vortex(wf, conf)
wf, LS_amp, LS_phase = lyotstop(wf, conf, RAVC=True, APP=False)

conf['ndet'] = int(np.ceil(2*conf['hfov']*1000/conf['pscale']))    
if conf['ndet'] %2 :
    conf['ndet'] += 1
psf = detector(wf, conf)

npupil = 251


In [3]:
plt.figure()
im, norm = imshow_norm(psf, plt.gca(), origin='lower',
                       interval=MinMaxInterval(),
                       stretch=LogStretch())
plt.colorbar(im)
plt.title('Post-coronagraph PSF')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Post-coronagraph PSF')